In [148]:
import numpy as np
import pandas as pd
import sklearn

import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [149]:
df = pd.read_csv('data/uci_data.csv')

In [150]:
df.head(10)

,Unnamed: 0,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,y
0,X21.V1.791,135,190,229,223,192,125,55,-9,-33,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,4
1,X15.V1.924,386,382,356,331,320,315,307,272,244,...,164,150,146,152,157,156,154,143,129,1
2,X8.V1.1,-32,-39,-47,-37,-32,-36,-57,-73,-85,...,57,64,48,19,-12,-30,-35,-35,-36,5
3,X16.V1.60,-105,-101,-96,-92,-89,-95,-102,-100,-87,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,5
4,X20.V1.54,-9,-65,-98,-102,-78,-48,-16,0,-21,...,4,2,-12,-32,-41,-65,-83,-89,-73,5
5,X14.V1.56,55,28,18,16,16,19,25,40,52,...,-12,-31,-42,-54,-60,-64,-60,-56,-55,5
6,X3.V1.191,-55,-9,52,111,135,129,103,72,37,...,-125,-99,-79,-62,-41,-26,11,67,128,4
7,X11.V1.273,1,-2,-8,-11,-12,-17,-15,-16,-18,...,-79,-91,-97,-88,-76,-72,-66,-57,-39,2
8,X19.V1.874,-278,-246,-215,-191,-177,-167,-157,-139,-118,...,-400,-379,-336,-281,-226,-174,-125,-79,-40,1
9,X3.V1.491,8,15,13,3,-6,-8,-5,4,25,...,49,31,11,-5,-17,-19,-15,-15,-11,4


In [151]:
print(df.info)

<bound method DataFrame.info of        Unnamed: 0   X1   X2   X3   X4   X5   X6   X7   X8   X9  ...  X170  \
0      X21.V1.791  135  190  229  223  192  125   55   -9  -33  ...   -17   
1      X15.V1.924  386  382  356  331  320  315  307  272  244  ...   164   
2         X8.V1.1  -32  -39  -47  -37  -32  -36  -57  -73  -85  ...    57   
3       X16.V1.60 -105 -101  -96  -92  -89  -95 -102 -100  -87  ...   -82   
4       X20.V1.54   -9  -65  -98 -102  -78  -48  -16    0  -21  ...     4   
...           ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   
11495  X22.V1.114  -22  -22  -23  -26  -36  -42  -45  -42  -45  ...    15   
11496  X19.V1.354  -47  -11   28   77  141  211  246  240  193  ...   -65   
11497    X8.V1.28   14    6  -13  -16   10   26   27   -9    4  ...   -65   
11498  X10.V1.932  -40  -25   -9  -12   -2   12    7   19   22  ...   121   
11499  X16.V1.210   29   41   57   72   74   62   54   43   31  ...   -59   

       X171  X172  X173  X174  X175  X176  

# Preprocessing our data

Before we can start running models on our data, we need to clean it up a bit.

- First, let's get rid of the different indicators in our 'y' column. All we care about is whether the patient is seizing or not --> We want to turn this into a binary classification problem.

In [152]:
df['output'] = df.y == 1
df['output'] = df['output'].astype(int)
df.pop('y')
df.drop(df.columns[0], axis=1, inplace=True)

In [153]:
df.head(5)

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X170,X171,X172,X173,X174,X175,X176,X177,X178,output
0,135,190,229,223,192,125,55,-9,-33,-38,...,-17,-15,-31,-77,-103,-127,-116,-83,-51,0
1,386,382,356,331,320,315,307,272,244,232,...,164,150,146,152,157,156,154,143,129,1
2,-32,-39,-47,-37,-32,-36,-57,-73,-85,-94,...,57,64,48,19,-12,-30,-35,-35,-36,0
3,-105,-101,-96,-92,-89,-95,-102,-100,-87,-79,...,-82,-81,-80,-77,-85,-77,-72,-69,-65,0
4,-9,-65,-98,-102,-78,-48,-16,0,-21,-59,...,4,2,-12,-32,-41,-65,-83,-89,-73,0


In [160]:
collist = df.columns.tolist()[:178]
df = df[collist + ['output']]
df.info

<bound method DataFrame.info of         X1   X2   X3   X4   X5   X6   X7   X8   X9  X10  ...  X170  X171  \
0      -10  -26   -5   13   32    7  -57 -108 -109 -104  ...    -3    -8   
1      658  537  392  258  119   -6 -119 -203 -269 -377  ...  -237  -297   
2       44   36   10  -17  -34  -32  -21  -13  -17  -28  ...    16     4   
3      -24  -28  -30  -36  -41  -52  -49  -45  -41  -36  ...   -31   -33   
4      -19  -37  -32  -39  -35  -21    1   24   18  -30  ...   -24   -30   
...    ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   ...   ...   
11495   89   91   86   78   75   65   62   64   67   70  ...    89   108   
11496  -69  -83  -86  -82  -80  -82  -89  -96  -93  -86  ...   -41   -42   
11497   85   52   41   41   60   69   87  116  125  119  ...    14     0   
11498 -205 -198 -189 -172 -158 -137 -112  -87  -67  -51  ...   -66   -73   
11499    0    7    6   14   12   15   16   12    4    0  ...    14    15   

       X172  X173  X174  X175  X176  X177  X178  output

In [161]:
df = df.sample(n=len(df))
df = df.reset_index(drop=True)
df.info

<bound method DataFrame.info of         X1   X2   X3   X4   X5   X6   X7    X8    X9   X10  ...  X170  X171  \
0     -238 -233 -255 -356 -303 -150  119   365   606   770  ... -1453 -1467   
1       90   93   83   69   52   41   31    21    23    18  ...    12    23   
2       81  404  780  930  900  438 -388 -1163 -1616 -1420  ...  -730 -1417   
3      105   90   82   73   67   52   52    49    53    48  ...    34    35   
4      -12  -14  -25  -32  -37  -37  -37   -26   -24   -19  ...     7     5   
...    ...  ...  ...  ...  ...  ...  ...   ...   ...   ...  ...   ...   ...   
11495  196  233  198   94 -108 -334 -507  -584  -574  -545  ...   -19  -139   
11496   -6   -3  -10   -6   -1    6   13    23    18    21  ...    55    61   
11497 -397 -395 -400 -399 -320 -154   79   311   454   482  ...    49   196   
11498   26   40   33   34   31   28   16    10    29    47  ...   -23   -23   
11499 -107 -103 -105 -107 -104 -102 -103  -104  -101   -99  ...   -53   -50   

       X172  X173  

Now, we'll want to split our data into a training, testing, and validation set, using a 70/15/15 way split.

In [91]:
validation_and_testing = df.sample(frac=0.3)
testing = validation_and_testing.sample(frac=0.5)
validation = validation_and_testing.drop(testing.index)
all_training = df.drop(validation_and_testing.index)

In [162]:
# Returns the frequency of positive labels
def freq(y):
    return sum(y) / len(y)

print('Testing dataset length: ', len(testing), ' frequency of positive samples: ', freq(testing['output'].values))
print('Validation dataset length: ', len(validation), ' frequency of positive samples: ', freq(validation['output'].values))
print('Training dataset length: ', len(all_training), ' frequency of positive samples: ', freq(all_training['output'].values))


Testing dataset length:  1725  frequency of positive samples:  0.20521739130434782
Validation dataset length:  1725  frequency of positive samples:  0.20173913043478262
Training dataset length:  8050  frequency of positive samples:  0.19850931677018632


It's clear that we have a minority of seizure examples in our dataset (people *usually* aren't seizing). However, that means that our model could learn to blindly classify everything as belonging to the majority class - non-seizure in this case. To fix that problem, we can down-sample the number of cases we have of non-seizure activity so they are more evenly balanced.

In [165]:
positive_class = all_training.output == 1
positive = all_training.loc[positive_class]
negative = all_training.loc[~positive_class]

# Merge balanced data
n = np.min([len(positive), len(negative)])
df_train = pd.concat([positive.sample(n=n), negative.sample(n=n)], axis=0, ignore_index=True)

# Shuffle it up
df_train = df_train.sample(n=len(df_train), random_state=42).reset_index(drop=True)
print('Training dataset length: ', len(df_train), ' frequency of positive samples: ', freq(df_train['output'].values))

Training dataset length:  3196  frequency of positive samples:  0.5


In [168]:
x_cols = df.columns.tolist()[:178]
X_train = df_train[x_cols].values
all_X_train = all_training[x_cols].values
X_valid = validation[x_cols].values

y_train = df_train['output'].values
y_valid = validation['output'].values

print('Training All shapes:', all_X_train.shape)
print('Training shapes:',X_train.shape, y_train.shape)
print('Validation shapes:',X_valid.shape, y_valid.shape)


Training All shapes: (8050, 178)
Training shapes: (3196, 178) (3196,)
Validation shapes: (1725, 178) (1725,)


## Before we start running our data through different models, we should also take care to scale it correctly.

In [186]:
from sklearn.preprocessing import StandardScaler

In [187]:
scaler = StandardScaler()
scaler.fit(all_X_train)
X_train_tf = scaler.transform(X_train)
X_valid_tf = scaler.transform(X_valid)

In [188]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(3196, 178) (3450, 178) (3196,) (3450,)


# Model Evaluation

Because we'll be evaluating a few different models, let's set up our evaluation framework early on.

In [255]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score, confusion_matrix

def specificity_score(y_pred, y_test, thresh=0.5):
    '''
    Returns probability of incorrectly classified negative samples (false positives)
    '''
    return sum((y_pred < thresh) & (y_test == 0)) / sum(y_test == 0)

def print_cm(cm, labels, hide_zeroes=False, hide_diagonal=False, hide_threshold=None):
    """pretty print for confusion matrixes"""
    columnwidth = max([len(x) for x in labels] + [5])  # 5 is value length
    empty_cell = " " * columnwidth
    
    # Begin CHANGES
    fst_empty_cell = (columnwidth-3)//2 * " " + "t/p" + (columnwidth-3)//2 * " "
    
    if len(fst_empty_cell) < len(empty_cell):
        fst_empty_cell = " " * (len(empty_cell) - len(fst_empty_cell)) + fst_empty_cell
    # Print header
    print("    " + fst_empty_cell, end=" ")
    # End CHANGES
    
    for label in labels:
        print("%{0}s".format(columnwidth) % label, end=" ")
        
    print()
    # Print rows
    for i, label1 in enumerate(labels):
        print("    %{0}s".format(columnwidth) % label1, end=" ")
        for j in range(len(labels)):
            cell = "%{0}.1f".format(columnwidth) % cm[i, j]
            if hide_zeroes:
                cell = cell if float(cm[i, j]) != 0 else empty_cell
            if hide_diagonal:
                cell = cell if i != j else empty_cell
            if hide_threshold:
                cell = cell if cm[i, j] > hide_threshold else empty_cell
            print(cell, end=" ")
        print()
        
def evaluate(y_pred, y_actual, error_threshold=0.5, model_name=''):
    '''
    Param: y_pred: Predicted values from model
    Param: y_actual: Actual results
    Param: error_threshold: Our threshold for determining Type I vs. Type II error
    Output: Printed summary of model performance
    '''
    auc = roc_auc_score(y_actual, y_pred)
    accuracy = accuracy_score(y_actual, (y_pred > error_threshold))
    recall = recall_score(y_actual, (y_pred > error_threshold))
    precision = precision_score(y_actual, (y_pred > error_threshold))
    specificity = specificity_score(y_pred, y_actual)
    
    print('************* Evaluating Model ****************')
    if model_name:
        print('Model: {}'.format(model_name))
    print('AUC: %.3f' % auc)
    print('Accuracy: %.3f'% accuracy)
    print('Recall: %.3f'% recall)
    print('Precision: %.3f'% precision)
    print('Specificity: %.3f'% specificity)
    
    cm = confusion_matrix(y_actual, (y_pred > error_threshold))
    print('Confusion Matrix:')
    print_cm(cm, ['Ictal', 'Interictal'])
    print('\n\n')
    

In [256]:
import warnings
warnings.filterwarnings('ignore')

# Modelling Seizure Activity

## Method \#1: Logistic Regression

Logistic Regression is at the heart of neural networks. It's ideal for two class discrimination problems, where continuous variables map to discrete outputs.

In [261]:
from sklearn.linear_model import LogisticRegression

log_rg = LogisticRegression(random_state=42).fit(X_train_tf, y_train)
y_train_pred = log_rg.predict_proba(X_train_tf)[:,1]
y_valid_pred = log_rg.predict_proba(X_valid_tf)[:,1]

evaluate(y_train_pred, y_train, error_threshold=0.45, model_name='Logistic Regression Test Set')
evaluate(y_valid_pred, y_valid, error_threshold=0.45, model_name='Logistic Regression Validation Set')

************* Evaluating Model ****************
Model: Logistic Regression Test Set
AUC: 0.636
Accuracy: 0.503
Recall: 0.695
Precision: 0.502
Specificity: 0.802
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal      495.0     1103.0 
    Interictal      487.0     1111.0 



************* Evaluating Model ****************
Model: Logistic Regression Validation Set
AUC: 0.547
Accuracy: 0.343
Recall: 0.618
Precision: 0.177
Specificity: 0.768
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal      376.0     1001.0 
    Interictal      133.0      215.0 





## Method 2: KNN
K-Nearest Neighbors is a classic clustering algorithm, that could deal well with the non-linearities found in seizure data. Let's see if it can perform better than simple linear regression

In [258]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train_tf, y_train)

y_train_pred = knn.predict_proba(X_train_tf)[:,1]
y_valid_pred = knn.predict_proba(X_valid_tf)[:,1]

evaluate(y_train_pred, y_train, error_threshold=0.45, model_name='KNN Test Set')
evaluate(y_valid_pred, y_valid, error_threshold=0.45, model_name='KNN Validation Set')

************* Evaluating Model ****************
Model: KNN Test Set
AUC: 0.994
Accuracy: 0.650
Recall: 0.300
Precision: 0.998
Specificity: 0.999
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal     1597.0        1.0 
    Interictal     1119.0      479.0 



************* Evaluating Model ****************
Model: KNN Validation Set
AUC: 0.966
Accuracy: 0.853
Recall: 0.282
Precision: 0.970
Specificity: 0.998
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal     1374.0        3.0 
    Interictal      250.0       98.0 





## Model 3: Multi-layer Perceptron

In [259]:
from sklearn.neural_network import MLPClassifier

mlp = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(10, 5), random_state=1)
mlp.fit(X_train_tf, y_train)

y_train_pred = mlp.predict_proba(X_train_tf)[:,1]
y_valid_pred = mlp.predict_proba(X_valid_tf)[:,1]

evaluate(y_train_pred, y_train, error_threshold=0.45, model_name='MLP Test Set')
evaluate(y_valid_pred, y_valid, error_threshold=0.45, model_name='MLP Validation Set')

************* Evaluating Model ****************
Model: MLP Test Set
AUC: 1.000
Accuracy: 1.000
Recall: 1.000
Precision: 1.000
Specificity: 1.000
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal     1598.0        0.0 
    Interictal        0.0     1598.0 



************* Evaluating Model ****************
Model: MLP Validation Set
AUC: 0.978
Accuracy: 0.934
Recall: 0.934
Precision: 0.781
Specificity: 0.935
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal     1286.0       91.0 
    Interictal       23.0      325.0 





## Model 4: Support Vector Machine

In [260]:
from sklearn import svm

clf = svm.SVC(gamma='auto', probability=True)
clf.fit(X_train_tf, y_train)

y_train_pred = clf.predict_proba(X_train_tf)[:,1]
y_valid_pred = clf.predict_proba(X_valid_tf)[:,1]

evaluate(y_train_pred, y_train, error_threshold=0.45, model_name='SVM Test Set')
evaluate(y_valid_pred, y_valid, error_threshold=0.45, model_name='SVM Validation Set')

************* Evaluating Model ****************
Model: SVM Test Set
AUC: 0.997
Accuracy: 0.981
Recall: 0.980
Precision: 0.982
Specificity: 0.982
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal     1569.0       29.0 
    Interictal       32.0     1566.0 



************* Evaluating Model ****************
Model: SVM Validation Set
AUC: 0.991
Accuracy: 0.962
Recall: 0.968
Precision: 0.860
Specificity: 0.964
Confusion Matrix:
        t/p         Ictal Interictal 
         Ictal     1322.0       55.0 
    Interictal       11.0      337.0 



